# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import os
import glob


#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

all_file_path = glob.glob(filepath+'/*.csv')


C:\Vivek\Workspace\learningprojects\BigData\Project_2_Cassandra_Data_Modeling_and_ETL


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Read the file using Pandas and create DataFrame.
data = []
for file in all_file_path:
    df = pd.read_csv(file, encoding='UTF-8')
    data.append(df)


frame = pd.DataFrame(pd.concat(data))

In [4]:
# Remove the null Value from the artist column 
frame.dropna(subset=['artist'],inplace=True)


In [5]:
# Create a column list whose data we need to write, which we will use for further processing
columns_to_write = ['artist', 'firstName', 'gender', 'itemInSession', 'lastName',
                    'length', 'level', 'location', 'sessionId', 'song', 'userId']

# write data to event_data_combined file,
frame.to_csv('event_data_combined.csv',columns=columns_to_write,encoding='utf8',index=False)

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_data_combined.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The data below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>


In [6]:
df = pd.read_csv('event_data_combined.csv')
df.head(5)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Des'ree,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",139,You Gotta Be,8.0
1,Mr Oizo,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",139,Flat 55,8.0
2,Tamba Trio,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",139,Quem Quiser Encontrar O Amor,8.0
3,The Mars Volta,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",139,Eriatarka,8.0
4,Infected Mushroom,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",139,Becoming Insane,8.0


#### Creating a Cluster

In [7]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    
    #To establish connection and begin executing queries, we need to create session
    session = cluster.connect()

except Exception as e:
    print('Error occured while connecting to the cassandra database')
    print(e)


#### Create Keyspace

In [8]:
# create a keyspace
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = { 'class':'SimpleStrategy','replication_factor':1 }
    """)
    print('Created keyspace')
except Exception as e:
    print('Erro occured while creating keyspace')
    print(e)

Created keyspace


#### Set Keyspace

In [9]:
session.set_keyspace('sparkify')

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Query #1

### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

To answer this question we will need to obtain (select) the artist name, song name, and song length from out table, and we will need to filter by sessionId and itemInSession.
In CQL our query looks like:

*SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4*

- We will name our table **session_songs**
- Our primary key will consist of partition key sessionId, and clustering key itemInSession so that we can filter by this attributes later on.
- The columns of our table will be: sessionId, itemInSession, artist, song_title and song_length.


In [10]:
# Create Table session_songs
session.execute("""CREATE TABLE IF NOT EXISTS session_songs 
    (sessionId int, itemInSession int, artist text, song_title text, song_length float, 
    PRIMARY KEY(sessionId, itemInSession))
  """)

In [11]:
# insert data to session table. 
insert_query = 'INSERT INTO session_songs (sessionId, itemInSession, artist, song_title , song_length)'
insert_query = insert_query + 'VALUES(%s, %s, %s, %s, %s)'


for index, row in df.iterrows():
    session.execute(insert_query,(int(row['sessionId']),int(row['itemInSession']),row['artist'],row['song'],float(row['length'])))
    



#### SELECT to verify that the data have been inserted into each table

In [12]:
rows = session.execute("""SELECT artist, song_title, song_length FROM session_songs 
                            WHERE sessionId = 338 AND itemInSession = 4""")

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Query #2

### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To answer this question we will need to obtain (select) the artist name, song name, user name and user lastname from out table, we will need to filter by userId and sessionId, and order by itemInSession. 

In CQL our query looks like:

*SELECT itemInSession, artist, song, firstName, lastName FROM user_songs WHERE userId = 10 AND sessionId = 182*

- We will name our table **user_songs**
- Our primary key will consist of composite partition key userId, sessionId. The reason for this is that if we only use userId as partition key, the sessionid which belongs to the same user will be put into different nodes, which will have the performance issue when the volume of data is large.
- Our clustering key will be itemInSession so that our results are order by it.
- The columns of our table will be: userId, sessionId, itemInSession, artist, song and firstName and lastName.


In [13]:
# Create Table user_songs
session.execute("""CREATE TABLE IF NOT EXISTS user_songs 
    (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, 
    PRIMARY KEY((userId, sessionId),itemInSession))
  """)

In [14]:
# insert data to user_songs. 
insert_query = 'INSERT INTO user_songs (userId,sessionId, itemInSession, artist, song , firstName, lastName)'
insert_query = insert_query + 'VALUES(%s, %s, %s, %s, %s, %s, %s)'


for index, row in df.iterrows():
    session.execute(insert_query,(int(row['userId']), int(row['sessionId']),
                                  int(row['itemInSession']),row['artist'],
                                  row['song'],row['firstName'],row['lastName']))
    

In [15]:
rows = session.execute("""SELECT itemInSession, artist, song, firstName, lastName FROM user_songs 
                    WHERE userId = 10 AND sessionId = 182""")

for row in rows:
    print(row.iteminsession, row.artist,row.song, row.firstname, row.lastname)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# Query #3

### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

To answer this question we will need to obtain (select) the user first name and lastname from out table, and we will need to filter by song name. As user name and lastname, in large datasets, are not unique, we will add the column userId to uniquely identify users.

In CQL our query looks like:

*SELECT firstName, lastName FROM app_history WHERE song = 'All Hands Against His Own'*

- We will name our table **app_history**
- Our primary key will consist of partition key song, and clustering key userId. This uniquely identifies our rows.
- The columns of our table will be: song, firstName, lastName and userId.


In [16]:
# Create Table app_history
session.execute("""CREATE TABLE IF NOT EXISTS app_history 
    (song text, firstName text, lastName text,userId int, 
    PRIMARY KEY(song,userId))
  """)

In [17]:
# insert data to app_history. 
insert_query = 'INSERT INTO app_history (song , firstName, lastName,userId)'
insert_query = insert_query + 'VALUES(%s, %s, %s, %s)'


for index, row in df.iterrows():
    session.execute(insert_query,(row['song'],row['firstName'],row['lastName'],int(row['userId'])))
    

In [18]:
# Execute the final Queries:
rows = session.execute("""SELECT firstName, lastName FROM app_history WHERE song = 'All Hands Against His Own'""")

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### We will drop the tables before closing out the sessions

In [19]:
# Lets drop all the tables created

session.execute("""DROP TABLE session_songs""")
session.execute("""DROP TABLE user_songs""")
session.execute("""DROP TABLE app_history""")


### Close the session and cluster connection¶

In [20]:
session.shutdown()

cluster.shutdown()